<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignmnet 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


In [8]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [14]:
def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=13, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mean_squared_error'])
    return model

In [ ]:
# model = create_model()
epochs = 2000
model.fit(train_data, train_targets, batch_size=32, epochs=epochs, validation_split=.1, verbose=1)

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [2]:
import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [34]:
x_train.shape

(60000, 28, 28)

In [35]:
x_test.shape

(10000, 28, 28)

In [3]:
X_train = x_train.reshape(60000, 784)
X_test = x_test.reshape(10000, 784)

In [37]:
y_train.shape

(60000,)

In [38]:
y_test.shape

(10000,)

In [4]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
def create_mnist_model():
    model = Sequential()
    model.add(Dense(64, input_dim=784, activation='relu'))
    model.add(Dense(48, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [8]:
# model = create_mnist_model()
epochs = 20
model.fit(X_train, y_train, batch_size=200, epochs=epochs, validation_split=.1, verbose=1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 0s 7us/sample - loss: 0.2890 - accuracy: 0.8951 - val_loss: 0.4100 - val_accuracy: 0.8715
Epoch 2/20
54000/54000 [==============================] - 0s 7us/sample - loss: 0.2839 - accuracy: 0.8963 - val_loss: 0.4139 - val_accuracy: 0.8675
Epoch 3/20
54000/54000 [==============================] - 0s 7us/sample - loss: 0.2891 - accuracy: 0.8964 - val_loss: 0.4351 - val_accuracy: 0.8545
Epoch 4/20
54000/54000 [==============================] - 0s 7us/sample - loss: 0.2868 - accuracy: 0.8960 - val_loss: 0.4082 - val_accuracy: 0.8713
Epoch 5/20
54000/54000 [==============================] - 0s 7us/sample - loss: 0.2786 - accuracy: 0.8988 - val_loss: 0.4152 - val_accuracy: 0.8673
Epoch 6/20
54000/54000 [==============================] - 0s 8us/sample - loss: 0.2796 - accuracy: 0.8976 - val_loss: 0.4280 - val_accuracy: 0.8655
Epoch 7/20
54000/54000 [==============================] - 0s 7u

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [12]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
from keras import optimizers
import tensorflow as tf

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

def cnn_mnist_model():
    model = Sequential()
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
#     model.summary()
    
    model.compile(loss='categorical_crossentropy',
             metrics=['accuracy'],
             optimizer='adam')
    return model
    
    

In [ ]:
model = cnn_mnist_model()

model.fit(x_train, y_train, batch_size=32, epochs=200)

Train on 60000 samples
Epoch 1/200
60000/60000 [==============================] - 30s 500us/sample - loss: 0.5568 - accuracy: 0.7972
Epoch 2/200
60000/60000 [==============================] - 32s 529us/sample - loss: 0.3934 - accuracy: 0.8566
Epoch 3/200
60000/60000 [==============================] - 31s 522us/sample - loss: 0.3488 - accuracy: 0.8728
Epoch 4/200
60000/60000 [==============================] - 32s 538us/sample - loss: 0.3257 - accuracy: 0.8812
Epoch 5/200
60000/60000 [==============================] - 34s 561us/sample - loss: 0.3076 - accuracy: 0.8875
Epoch 6/200
60000/60000 [==============================] - 34s 575us/sample - loss: 0.2944 - accuracy: 0.8915
Epoch 7/200
60000/60000 [==============================] - 35s 588us/sample - loss: 0.2863 - accuracy: 0.8945
Epoch 8/200
60000/60000 [==============================] - 34s 569us/sample - loss: 0.2735 - accuracy: 0.8982
Epoch 9/200
60000/60000 [==============================] - 39s 643us/sample - loss: 0.2690 - accu

In [14]:
scores = model.evaluate(x_test, y_test)
print(f'{model.metrics_names[1]}: {scores[1]*100}')

10000/10000 [==============================] - 3s 275us/sample - loss: 0.2251 - accuracy: 0.9310
accuracy: 93.09999942779541
